In [5]:
import tqdm as notebook_tqdm
import numpy as np
from dotenv import load_dotenv
import os
load_dotenv()
pc_key = os.getenv("PINECONEDB")

In [6]:
file_path = "C:\\Users\\ARYAN SURI\\Desktop\\Amlgo Labs Assignment\\data\\AI Training Document.pdf"


## Loading document using Langchain document_loader

In [7]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path)
docs = loader.load()

In [8]:
docs

[Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-06-18T14:59:12+05:30', 'author': 'Shivani Gupta', 'moddate': '2025-06-18T14:59:12+05:30', 'source': 'C:\\Users\\ARYAN SURI\\Desktop\\Amlgo Labs Assignment\\data\\AI Training Document.pdf', 'total_pages': 20, 'page': 0, 'page_label': '1'}, page_content='User Agreement \n1. Introduction \nThis User Agreement, the Mobile Application Terms of Use, and all policies and additional terms \nposted on and in our sites, applications, tools, and services (collectively "Services") set out the terms \non which eBay offers you access to and use of our Services. You can find an overview of our policies \nhere. The Mobile Application Terms of Use, all policies, and additional terms posted on and in our \nServices are incorporated into this User Agreement. You agree to comply with all terms of this User \nAgreement when accessing or using our Services. \nThe entity you are contracting with i

## Splitting test using RecursiveCharacterTextSplitter

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
documents  = text_splitter.split_documents(docs)
documents[:5]

[Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-06-18T14:59:12+05:30', 'author': 'Shivani Gupta', 'moddate': '2025-06-18T14:59:12+05:30', 'source': 'C:\\Users\\ARYAN SURI\\Desktop\\Amlgo Labs Assignment\\data\\AI Training Document.pdf', 'total_pages': 20, 'page': 0, 'page_label': '1'}, page_content='User Agreement \n1. Introduction \nThis User Agreement, the Mobile Application Terms of Use, and all policies and additional terms \nposted on and in our sites, applications, tools, and services (collectively "Services") set out the terms'),
 Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-06-18T14:59:12+05:30', 'author': 'Shivani Gupta', 'moddate': '2025-06-18T14:59:12+05:30', 'source': 'C:\\Users\\ARYAN SURI\\Desktop\\Amlgo Labs Assignment\\data\\AI Training Document.pdf', 'total_pages': 20, 'page': 0, 'page_label': '1'}, page_content='on which eBay offers you a

In [10]:
documents[0].page_content

'User Agreement \n1. Introduction \nThis User Agreement, the Mobile Application Terms of Use, and all policies and additional terms \nposted on and in our sites, applications, tools, and services (collectively "Services") set out the terms'

In [11]:
texts = [doc.page_content for doc in documents]

In [12]:
len(texts)

273

## Saving Chunks in chunks folder in JSON format

In [13]:
import json

with open("C:\\Users\\ARYAN SURI\Desktop\\Amlgo Labs Assignment\\chunks\\chunks.json", "w", encoding="utf-8") as f:
    json.dump(texts, f, ensure_ascii=False, indent=2)

## Using sentence embeddings to generate sentence embeddings

In [14]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(texts)

c:\Users\ARYAN SURI\.conda\envs\amlgolabs\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


## Making Vectordb using FAISS Indexing

In [15]:
# Convert to float32 (FAISS requirement)
faiss_embeddings = np.array(embeddings).astype("float32")

In [16]:
import faiss
index = faiss.IndexFlatL2(faiss_embeddings.shape[1])
index.add(faiss_embeddings)

In [17]:
faiss.write_index(index, "vectordb/faissIndex.faiss")

## Looking at Top 5 queries from FAISS Index

In [18]:
def search_chunks(query, top_k=5):
    # Encode the query using the same model
    query_embedding = model.encode([query])
    query_embedding = np.array(query_embedding).astype("float32")
    # Search the FAISS index
    D, I = index.search(query_embedding, top_k)
    # Collect results
    results = []
    for i, idx in enumerate(I[0]):
        result = {
            "chunk": texts[idx],
            "distance": float(D[0][i])
        }
        results.append(result)
    return results

In [19]:
query = "Address of the eBay contacted if residing in European Union?"
results = search_chunks(query, top_k=5)
for i, res in enumerate(results, 1):
    print(f"Result {i}:")
    print(f"Chunk: {res['chunk']}")
    print(f"Distance: {res['distance']}\n")

Result 1:
Chunk: any other country. In this User Agreement, these entities are individually and collectively referred to 
as "eBay," "we," or "us." 
If you reside in India and you register for our Services, you further agree to the eBay.in User 
Agreement.
Distance: 0.7906412482261658

Result 2:
Chunk: Draper, UT 84020. eBay will send any Notice to you to the physical address we have on file 
associated with your eBay account; it is your responsibility to keep your physical address up 
to date. To be valid, you must personally sign the Notice and complete all information on the
Distance: 0.8273691534996033

Result 3:
Chunk: eBay may contact you using autodialed or prerecorded calls and text messages, at any telephone 
number that you have provided us, to: (i) notify you regarding your account; (ii) troubleshoot 
problems with your account; (iii) resolve a dispute; (iv) collect a debt; (v) poll your opinions through
Distance: 0.8355653285980225

Result 4:
Chunk: Privacy Notice. These se

In [48]:
from langchain.vectorstores import FAISS
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
db = FAISS.from_documents(documents, 
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

## VectorSearch DB using pinecone

In [20]:
from pinecone import Pinecone

pc = Pinecone(api_key=pc_key)

In [21]:
from pinecone import ServerlessSpec

index_name = "amlgolabs"  # change if desired

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=89,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

In [22]:
from langchain_community.embeddings import HuggingFaceEmbeddings
pinecone_embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\ARYAN SURI\AppData\Local\Temp\ipykernel_88564\4165911852.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  pinecone_embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [23]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=pinecone_embedding)

In [24]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)

c:\Users\ARYAN SURI\.conda\envs\amlgolabs\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


['915855bd-f4e9-41b9-a057-75e9db2b32c2',
 '5d957bf4-0d1a-4733-86c5-a6229be69560',
 '78ccc1c5-2ce2-4fd5-b690-d7008dadcdca',
 'b1a7e5f8-47e1-4fd2-b1d9-f14e1f9c0b72',
 '2e8124e3-a659-4c16-8adf-c8f9068e486c',
 'a5e6a07b-6aee-4658-a5b2-fd0407585f0d',
 '88130741-b643-4a86-ad8c-413aa05950a3',
 '19037f09-c4bd-4b26-aac4-afe6d0944763',
 'c88fb76d-8e79-483c-96fb-f7780b4db785',
 '126b94f4-f185-48fb-88a2-da354cddb6fa',
 '1af7af5a-ad1f-429c-9adc-6a72eec92190',
 'bdceadd6-2f8b-443b-8a6b-a59b06593564',
 '73775345-8b40-4309-8046-3f6c6811c839',
 '459ad104-5bd0-44d4-a9bf-571928fc00e0',
 '7a575d56-4abe-423c-814c-6aa06b5b0222',
 'b270611c-cd7e-470e-890f-281738c1b99e',
 '4b2798e1-e0b9-4a86-be42-db45b8fc70e7',
 '0f4634af-1697-4cf7-801b-6e5cb991ad9b',
 '892d4ff5-6f74-4e32-8fe8-79cca143ab15',
 '1d01108c-938e-4f47-b8b0-84f2e3182bc0',
 '2d901cc4-a92e-4917-83d6-833bcb6e9f49',
 'e08d4d0c-b44c-4093-864d-8c71bea8cd68',
 'fdf3239a-7309-4699-a668-73a4774fe4c3',
 'ae1fd863-0dd6-4b89-a61b-04a3e95e20e2',
 'bfab5edd-eb15-

In [25]:
results = vector_store.similarity_search(
    "Address of the eBay contacted if residing in European Union?",
    k=2
)
for res in results:
    print(f"* {res.page_content}")

* Kingdom, if you reside in the United Kingdom; eBay GmbH, Albert-Einstein-Ring 2-6, 14532 
Kleinmachnow, Germany, if you reside in the European Union; eBay Canada Limited, 240 Richmond 
Street West, 2nd Floor Suite 02-100, Toronto, ON, M5V 1V6, Canada, if you reside in Canada; eBay 
Singapore Services Private Limited, 1 Raffles Quay, #18- 00, Singapore 048583, if you reside in India; 
and eBay Marketplaces GmbH, Helvetiastrasse 15/17, CH-3005, Bern, Switzerland, if you reside in 
any other country. In this User Agreement, these entities are individually and collectively referred to 
as "eBay," "we," or "us." 
If you reside in India and you register for our Services, you further agree to the eBay.in User 
Agreement. 
Read this User Agreement carefully as it contains provisions that govern how claims you and we have 
against each other are resolved (see "Disclaimer of Warranties; Limitation of Liability" and "Legal
* any other country. In this User Agreement, these entities are individu

## Using Mistral on above retriever(pinecone/faiss) as the generator

In [1]:
import transformers


model_name='mistralai/Mistral-7B-Instruct-v0.1'

model_config = transformers.AutoConfig.from_pretrained(
    model_name,
)

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [2]:
# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [3]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Your GPU supports bfloat16: accelerate training with bf16=True


In [27]:
model_llm = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
query = "Address of the eBay contacted if residing in European Union?"

In [71]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.chains import LLMChain

text_generation_pipeline = transformers.pipeline(
    model=model_llm,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=300,
)

prompt_template = """
### [INST]
You are a helpful assistant. Use the information from the context below to answer the user's question.
If the answer is not in the context, say "The document does not contain that information."

Context:
{context}

Question: {question}
[/INST]
"""


mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create prompt from prompt template 
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain 
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

Device set to use cuda:0


In [80]:
retriever = db.as_retriever()
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [81]:
from langchain_core.runnables import RunnablePassthrough
query = "Tell me about eBay?"
rag_chain = ( 
 {"context": retriever|format_docs, "question": RunnablePassthrough()}
    | llm_chain
)
answer = rag_chain.invoke(query)

In [82]:
answer

{'context': 'trial. \n2. About eBay \neBay is a marketplace that allows users to offer, sell, and buy goods and services in various \ngeographic locations using a variety of pricing formats. eBay is not a party to contracts for sale \nbetween third-party sellers and buyers, nor is eBay a traditional auctioneer.\n\nPayments for goods and services sold using our Services are facilitated by designated eBay entities \n(each, an "eBay Payment Entity") pursuant to the Payments Terms of Use. You agree to the Payments\n\neBay\'s applications may not occur in real time. Such functionality is subject to delays beyond eBay\'s \ncontrol. \nYOU AGREE THAT YOU ARE MAKING USE OF OUR SERVICES AT YOUR OWN RISK, AND THAT THEY ARE \nBEING PROVIDED TO YOU ON AN "AS IS" AND "AS AVAILABLE" BASIS. ACCORDINGLY , TO THE EXTENT\n\nconversations you or anyone acting on your behalf has with eBay or its agents for quality control and \ntraining purposes, or for its own protection. \neBay\'s automated systems scan 

In [83]:
print(answer['text'] )


### [INST]
You are a helpful assistant. Use the information from the context below to answer the user's question.
If the answer is not in the context, say "The document does not contain that information."

Context:
trial. 
2. About eBay 
eBay is a marketplace that allows users to offer, sell, and buy goods and services in various 
geographic locations using a variety of pricing formats. eBay is not a party to contracts for sale 
between third-party sellers and buyers, nor is eBay a traditional auctioneer.

Payments for goods and services sold using our Services are facilitated by designated eBay entities 
(each, an "eBay Payment Entity") pursuant to the Payments Terms of Use. You agree to the Payments

eBay's applications may not occur in real time. Such functionality is subject to delays beyond eBay's 
control. 
YOU AGREE THAT YOU ARE MAKING USE OF OUR SERVICES AT YOUR OWN RISK, AND THAT THEY ARE 
BEING PROVIDED TO YOU ON AN "AS IS" AND "AS AVAILABLE" BASIS. ACCORDINGLY , TO THE EXTE

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextIteratorStreamer, BitsAndBytesConfig
import threading
import torch
# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Important
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # Use BitsAndBytes config if quantized
    trust_remote_code=True,
    quantization_config = bnb_config
)

# Prepare prompt
prompt = "[INST] What is eBay’s refund policy? [/INST]"
inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(model.device)

# Prepare streamer
streamer = TextIteratorStreamer(tokenizer, skip_special_tokens=True)

# Generation args
generate_kwargs = dict(
    inputs=inputs.input_ids,
    attention_mask=inputs.attention_mask,  
    streamer=streamer,
    max_new_tokens=300
)

# Start generation in background thread
thread = threading.Thread(target=model.generate, kwargs=generate_kwargs)
thread.start()

# Stream tokens as they arrive
for token in streamer:
    print(token, end="", flush=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] What is eBay’s refund policy? [/INST] eBay's refund policy varies depending on the item and seller, but generally, if you are not satisfied with an item you purchased on eBay, you may be eligible for a refund or exchange. Here are the general steps to follow:

1. Contact the seller: First, try to contact the seller to resolve any issues with the item. Many sellers have their own refund or exchange policies, so it's important to check with them first.

2. Open a dispute: If you are unable to resolve the issue with the seller, you can open a dispute with eBay. eBay will review the case and determine if a refund or exchange is appropriate.

3. Provide evidence: When opening a dispute, be sure to provide evidence to support your claim. This can include photos of the item, a copy of the seller's refund policy, and any other relevant information.

4. Wait for a decision: eBay will review your case and make a decision. If your dispute is approved, eBay will process a refund or exchange